# LLM Iteration 10

**Aim**

Change wait time of 0.0 to always be positive. Put more functions inside the StrokeWard class.

**Prompt**

Update this model to generate patient arrivals and run the model inside the strokeward class. The wait times should be output as positive numbers only. 

**Note**

Modifications have been made to the output from the LLM, indicated by:
```python
########## MODIFICATION ##########
change
##################################
```
This is either used to add in code from previous iterations if only certain sections were updated, used to correct the LLM's code if required, or used to check the model is working as expected.

**LLM Output**

In [1]:
import simpy
import numpy as np
import math

In [2]:
class StrokeWard:
    def __init__(self, env, num_beds):
        self.env = env
        self.beds = simpy.Resource(env, capacity=num_beds)
        self.patient_count = 0
        self.wait_times = []
        self.length_of_stay = {
            "Stroke": {"mean": 7.4, "std_dev": 8.6},
            "TIA": {"mean": 1.8, "std_dev": 2.3},
            "Complex Neurological": {"mean": 4.0, "std_dev": 5.0},
            "Other": {"mean": 3.8, "std_dev": 5.2}
        }

        ########## MODIFICATION ##########
        self.patients_in_beds = 0
        self.bed_in_use_store = []
        self.los_store = []
        self.patient_type_store = []
        ##################################

    def admit_patient(self, patient):
        arrival_time = round(self.env.now, 2)
        with self.beds.request() as bed:
            yield bed
            ########## MODIFICATION ##########
            bed_arrival_time = self.env.now
            self.patients_in_beds += 1
            self.bed_in_use_store.append(self.patients_in_beds)
            self.patient_type_store.append(patient.patient_type)
            ##################################

            wait_time = max(0, round(self.env.now - arrival_time, 2))
            self.wait_times.append(wait_time)
            print(f'Patient {patient.id} ({patient.patient_type}) admitted to '
                  f'the stroke ward at day {round(self.env.now, 2)}. '
                  f'Wait time: {wait_time} days')
            mu, sigma = calculate_mu_sigma(self.length_of_stay[patient.\
                                           patient_type]["mean"],
                                           self.length_of_stay[patient.\
                                           patient_type]["std_dev"])
            los = np.random.default_rng(patient.seed).lognormal(mu, sigma)
            # Length of stay based on patient type
            yield self.env.timeout(round(los, 2))
            ########## MODIFICATION ##########
            self.patients_in_beds -= 1
            bed_leave_time = self.env.now
            self.los_store.append(round(bed_leave_time - bed_arrival_time, 2))
            ##################################
            print(f'Patient {patient.id} ({patient.patient_type}) discharged '
                  f'from the stroke ward at day {round(self.env.now, 2)}')

    def patient_arrival(self, patient_type, rng):
        while True:
            # Default interarrival time
            interarrival_time = rng.exponential(1.2)
            yield self.env.timeout(interarrival_time)
            patient = Patient(self.patient_count, patient_type, rng)
            self.patient_count += 1
            self.env.process(self.admit_patient(patient))

class Patient:
    def __init__(self, id, patient_type, seed):
        self.id = id
        self.patient_type = patient_type
        self.seed = seed

def calculate_mu_sigma(mean, std_dev):
    v = std_dev ** 2
    m = mean
    phi = math.sqrt(v + m ** 2)
    mu = math.log((m ** 2) / phi)
    sigma = math.sqrt(math.log((phi ** 2) / (m ** 2)))
    return mu, sigma

In [3]:
env = simpy.Environment()
stroke_ward = StrokeWard(env, num_beds=10)

# Set random seed for reproducibility for each patient type
random_seeds = {
    "Stroke": 12345,
    "TIA": 54321,
    "Complex Neurological": 98765,
    "Other": 67890
}

rngs = {patient_type: np.random.default_rng(seed) for patient_type,
        seed in random_seeds.items()}

for patient_type in random_seeds.keys():
    env.process(stroke_ward.patient_arrival(patient_type, rngs[patient_type]))

# Set the run length to 2 months (default)
run_length_days = round(60 * 2)

print("Simulation started...")
env.run(until=run_length_days)
print("Simulation ended.")

# Calculate and print the mean wait time for a bed rounded to two decimal
# places
mean_wait_time = round(np.mean(stroke_ward.wait_times), 2)
print(f"Mean wait time for a bed: {mean_wait_time} days")

Simulation started...
Patient 0 (Complex Neurological) admitted to the stroke ward at day 0.03. Wait time: 0 days
Patient 1 (Stroke) admitted to the stroke ward at day 0.22. Wait time: 0 days
Patient 2 (TIA) admitted to the stroke ward at day 0.26. Wait time: 0 days
Patient 0 (Complex Neurological) discharged from the stroke ward at day 0.43
Patient 3 (Stroke) admitted to the stroke ward at day 0.99. Wait time: 0 days
Patient 4 (Stroke) admitted to the stroke ward at day 1.5. Wait time: 0 days
Patient 5 (Complex Neurological) admitted to the stroke ward at day 1.76. Wait time: 0 days
Patient 6 (Other) admitted to the stroke ward at day 1.89. Wait time: 0 days
Patient 7 (TIA) admitted to the stroke ward at day 1.95. Wait time: 0 days
Patient 8 (Other) admitted to the stroke ward at day 2.05. Wait time: 0 days
Patient 9 (TIA) admitted to the stroke ward at day 2.09. Wait time: 0 days
Patient 10 (TIA) admitted to the stroke ward at day 2.11. Wait time: 0 days
Patient 7 (TIA) discharged fr

**Checks**

In [4]:
class TestCase:
    def __init__(self, env, ward, rngs):
        self.env = env
        self.ward = ward
        for patient_type in rngs.keys():
            env.process(self.ward.patient_arrival(patient_type,
                                                  rngs[patient_type]))

    def test_run_length(self):
        '''
        Checks that specified run length is implemented correctly.
        '''
        self.env.run(until=RUN_LENGTH)

        if self.env.now > RUN_LENGTH:
            print('The simulation has continued for longer than input.')

    def test_bed_hold(self):
        '''
        Checks that patients are holding beds until they leave the model.
        '''
        self.env.run(until=RUN_LENGTH)

        for bed_use in ward.bed_in_use_store:
            if bed_use > NUM_BEDS:
                print('The number of beds in use exceeded limit.')

    def test_length_of_stay(self):
        '''
        Checks that specified length of stay correctly implemented.
        '''
        self.env.run(until=RUN_LENGTH)

        print('\nLength of stay:')
        for pat_type, los in zip(ward.patient_type_store, ward.los_store):
            print(f'({pat_type}) {los}')

In [5]:
# Checks run length
NUM_BEDS = 10
RUN_LENGTH = 10

random_seeds = {
    "Stroke": 12345,
    "TIA": 54321,
    "Complex Neurological": 98765,
    "Other": 67890
}

rngs = {patient_type: np.random.default_rng(seed) for patient_type, \
        seed in random_seeds.items()}

env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS)

test = TestCase(env, ward, rngs)
test.test_run_length()

Patient 0 (Complex Neurological) admitted to the stroke ward at day 0.03. Wait time: 0 days
Patient 1 (Stroke) admitted to the stroke ward at day 0.22. Wait time: 0 days
Patient 2 (TIA) admitted to the stroke ward at day 0.26. Wait time: 0 days
Patient 0 (Complex Neurological) discharged from the stroke ward at day 0.43
Patient 3 (Stroke) admitted to the stroke ward at day 0.99. Wait time: 0 days
Patient 4 (Stroke) admitted to the stroke ward at day 1.5. Wait time: 0 days
Patient 5 (Complex Neurological) admitted to the stroke ward at day 1.76. Wait time: 0 days
Patient 6 (Other) admitted to the stroke ward at day 1.89. Wait time: 0 days
Patient 7 (TIA) admitted to the stroke ward at day 1.95. Wait time: 0 days
Patient 8 (Other) admitted to the stroke ward at day 2.05. Wait time: 0 days
Patient 9 (TIA) admitted to the stroke ward at day 2.09. Wait time: 0 days
Patient 10 (TIA) admitted to the stroke ward at day 2.11. Wait time: 0 days
Patient 7 (TIA) discharged from the stroke ward at 

In [6]:
# Checks that is only ever 1 patient if there is only 1 bed
NUM_BEDS = 1
RUN_LENGTH = 60

random_seeds = {
    "Stroke": 12345,
    "TIA": 54321,
    "Complex Neurological": 98765,
    "Other": 67890
}

rngs = {patient_type: np.random.default_rng(seed) for patient_type, \
        seed in random_seeds.items()}

env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS)

test = TestCase(env, ward, rngs)
test.test_bed_hold()

Patient 0 (Complex Neurological) admitted to the stroke ward at day 0.03. Wait time: 0 days
Patient 0 (Complex Neurological) discharged from the stroke ward at day 0.43
Patient 1 (Stroke) admitted to the stroke ward at day 0.43. Wait time: 0.21 days
Patient 1 (Stroke) discharged from the stroke ward at day 2.59
Patient 2 (TIA) admitted to the stroke ward at day 2.59. Wait time: 2.33 days
Patient 2 (TIA) discharged from the stroke ward at day 4.57
Patient 3 (Stroke) admitted to the stroke ward at day 4.57. Wait time: 3.58 days
Patient 3 (Stroke) discharged from the stroke ward at day 13.36
Patient 4 (Stroke) admitted to the stroke ward at day 13.36. Wait time: 11.86 days
Patient 4 (Stroke) discharged from the stroke ward at day 24.47
Patient 5 (Complex Neurological) admitted to the stroke ward at day 24.47. Wait time: 22.71 days
Patient 5 (Complex Neurological) discharged from the stroke ward at day 26.2
Patient 6 (Other) admitted to the stroke ward at day 26.2. Wait time: 24.31 days
Pa

In [7]:
# Checks length of stay for patients
NUM_BEDS = 10
RUN_LENGTH = 10

random_seeds = {
    "Stroke": 12345,
    "TIA": 54321,
    "Complex Neurological": 98765,
    "Other": 67890
}

rngs = {patient_type: np.random.default_rng(seed) for patient_type, \
        seed in random_seeds.items()}

env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS)

test = TestCase(env, ward, rngs)
test.test_length_of_stay()

Patient 0 (Complex Neurological) admitted to the stroke ward at day 0.03. Wait time: 0 days
Patient 1 (Stroke) admitted to the stroke ward at day 0.22. Wait time: 0 days
Patient 2 (TIA) admitted to the stroke ward at day 0.26. Wait time: 0 days
Patient 0 (Complex Neurological) discharged from the stroke ward at day 0.43
Patient 3 (Stroke) admitted to the stroke ward at day 0.99. Wait time: 0 days
Patient 4 (Stroke) admitted to the stroke ward at day 1.5. Wait time: 0 days
Patient 5 (Complex Neurological) admitted to the stroke ward at day 1.76. Wait time: 0 days
Patient 6 (Other) admitted to the stroke ward at day 1.89. Wait time: 0 days
Patient 7 (TIA) admitted to the stroke ward at day 1.95. Wait time: 0 days
Patient 8 (Other) admitted to the stroke ward at day 2.05. Wait time: 0 days
Patient 9 (TIA) admitted to the stroke ward at day 2.09. Wait time: 0 days
Patient 10 (TIA) admitted to the stroke ward at day 2.11. Wait time: 0 days
Patient 7 (TIA) discharged from the stroke ward at 

**Reflection**

As requested, the wait times are now always positive. The patient arrivals are now generated inside the StrokeWard class, meaning the number of functions has been reduced as there is now not a separate arrival process for different patient types. The LLM did not include an option to run the model inside this class as instructed. However, the LLM has changed the interarrival time for all patients to have a mean of 1.2, which is the IAT for Stroke type patients only, calling this a 'Default interarrival time' (interarrival_time = rng.exponential(1.2)).